Реализация SBF (стандратного фильтра Блума) и VBF.

Для начала реализуем SBF:

In [43]:
import zlib

class SBF():
    def __init__(self, size, hash_func=None):
        self.size = size
        self.bit_array = [0] * size
        self.hash_func = [zlib.crc32]

    def add(self, item):
        for hash_f in self.hash_func:
            ind = hash_f(item.encode()) % self.size
            self.bit_array[ind] = 1

    def check(self, item):
        for hash_f in self.hash_func:
            ind = hash_f(item.encode()) % self.size
            if self.bit_array[ind] == 0:
                return False
            return True


Тут была использованна только хэш-функция crc32, но мы можем увеличить количетсво хэш-функций для минимизации ложно-положительных результатов

In [74]:
import zlib

def some_another_hash(x):
    s = 0
    for i in x:
        s += int(i)
    return s

def some_another_hash2(x):
    s = 0
    for i in x:
        s += int(i) ** 2
    return s

class SBF():
    def __init__(self, size, hash_func=None):
        self.size = size
        self.bit_array = [0] * size
        self.hash_func = [zlib.crc32, some_another_hash,some_another_hash2]

    def add(self, item):
        for hash_f in self.hash_func:
            ind = hash_f(item.encode()) % self.size
            self.bit_array[ind] = 1


    def check(self, item):
        for hash_f in self.hash_func:
            ind = hash_f(item.encode()) % self.size
            if self.bit_array[ind] == 0:
                return False
        return True

Но можно поступить проще и обойтись лишь одной хэш-функцией: будет добавлять i к хэш-функции

Хоть функции удаления нет в SBF, добавим ее. Это нужно для сранения с VBF

In [310]:
import zlib

class SBF():
    def __init__(self, size, count_hash_func=1):
        self.size = size
        self.bit_array = [0] * size
        self.num_hash_func = count_hash_func
    
    def hash_func(self, item, number):
        return zlib.crc32((item+str(number)).encode())

    def add(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] = 1

    def delete(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] = 0
        
    def check(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            if self.bit_array[ind] == 0:
                return False
        return True

Теперь можно проверить работу: \
для наглядности возьмем пример проверки нахождения ip-адреса в черном списке

In [311]:
bf = SBF(100, 5)

ban_ip = "123.423.53.12"

bf.add(ban_ip)

for i in range(100):
    if bf.check(f'123.423.53.{i}'):
        print(f'Found: ' f'123.423.53.{i}')


Found: 123.423.53.12


Теперь реализуем VBF

In [312]:
class VBF():
    def __init__(self, size: int, count_hash_func: int, d: int, t: int, q: int):
        self.size = size
        self.bit_array = [0] * size
        self.num_hash_func = count_hash_func
        self.d = d
        self.t = t
        self.q = q

    def hash_func(self, item, number):
        return zlib.crc32((item+str(number)).encode())

    def add(self, item):
        for ind_hash in range(self.t):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] = 1

    def check(self, item):
        matched = 0
        
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            matched += self.bit_array[ind]
        
        return matched >= self.q

    def delete(self, item):
        deleted = 0
        if self.num_hash_func - self.d == 0:
            return

        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            if self.bit_array[ind]:
                self.bit_array[ind] = 0
                deleted += 1
                if deleted >= self.num_hash_func - self.d:
                    break
    

Также проверим работоспособность:

In [313]:
bf = VBF(100, 11, 0, 11, 10)

ban_ip = "123.423.53.12"
bf.add(ban_ip)

for i in range(100):
    if bf.check(f'123.423.53.{i}'):
        print(f'Found: ' f'123.423.53.{i}')

Found: 123.423.53.12


Теперь попытаемcя воспроизвести числовые данные из статьи

In [416]:
import random
import string

vbf = VBF(10000, 3, 0, 3, 2)
sbf = SBF(10000, 3)

Сгенерируем случайные данные

In [410]:
def make_random_str():
    alph = string.ascii_lowercase
    s = ''.join(random.choice(alph) for _ in range(20))
    return s

generate_data = [make_random_str() for _ in range(5000000)]

Пусть первые 25000 находятся в фильтре

In [417]:
for i in range(25000):
    vbf.add(generate_data[i])
    sbf.add(generate_data[i])

Удалим 10% из фильтра

In [418]:
for i in range(2500):
    vbf.delete(generate_data[i])
    sbf.delete(generate_data[i])

Теперь посчитаем FP, FN

In [419]:
vbf_F = [0, 0]
sbf_F = [0, 0]


for i in range(2500):
    vbf_F[1] += vbf.check(generate_data[i])
    sbf_F[1] += sbf.check(generate_data[i])

for i in range(2500, 25000):
    vbf_F[0] += not vbf.check(generate_data[i])
    sbf_F[0] += not sbf.check(generate_data[i])

for i in range(25000, 1600000):
    vbf_F[1] += vbf.check(generate_data[i])
    sbf_F[1] += sbf.check(generate_data[i])

In [420]:
print(f'FN VBF:{vbf_F[0]} \nFP VBF:{vbf_F[1]}')

FN VBF:12350 
FP VBF:717170


In [421]:
print(f'FN SBF:{sbf_F[0]} \nFP SBF:{sbf_F[1]}')

FN SBF:20120 
FP SBF:163941


Как видно из данных, FN действительно стал ниже при использовании VBF.

Реализуем счетные SBF и VBF

In [393]:
class CSBF():
    def __init__(self, size, count_hash_func=1):
        self.size = size
        self.bit_array = [0] * size
        self.num_hash_func = count_hash_func
    
    def hash_func(self, item, number):
        return zlib.crc32((item+str(number)).encode())

    def add(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] += 1

    def delete(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] -= 1
        
    def check(self, item):
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            if self.bit_array[ind] <= 0:
                return False
        return True

In [394]:
class CVBF():
    def __init__(self, size: int, count_hash_func: int, d: int, t: int, q: int):
        self.size = size
        self.bit_array = [0] * size
        self.num_hash_func = count_hash_func
        self.d = d
        self.t = t
        self.q = q

    def hash_func(self, item, number):
        return zlib.crc32((item+str(number)).encode())

    def add(self, item):
        for ind_hash in range(self.t):
            ind = self.hash_func(item, ind_hash) % self.size
            self.bit_array[ind] += 1

    def check(self, item):
        matched = 0
        
        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            matched += self.bit_array[ind]
        
        return matched >= self.q

    def delete(self, item):
        deleted = 0
        if self.num_hash_func - self.d == 0:
            return

        for ind_hash in range(self.num_hash_func):
            ind = self.hash_func(item, ind_hash) % self.size
            if self.bit_array[ind]:
                self.bit_array[ind] -= 1
                deleted += 1
                if deleted >= self.num_hash_func - self.d:
                    break
    